In [1]:
# https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
# https://codeburst.io/web-scraping-101-with-python-beautiful-soup-bb617be1f486
# https://github.com/CrisGCO17/online-popularity-machine-learning/

In [138]:
# importing necessary libraries
import requests
import bs4
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import urllib.request as ul
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_predict, cross_val_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

# silencing the warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# getting the stopwords from the ntlk library
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhivyaslh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# loading the dataset
data = pd.read_csv("OnlineNewsPopularity.csv")
data

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.700000,-0.350000,-0.600000,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.700000,-0.118750,-0.125000,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.000000,-0.466667,-0.800000,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.800000,-0.369697,-0.600000,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.000000,-0.220192,-0.500000,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.600000,-0.195000,-0.400000,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.000000,-0.224479,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.000000,-0.242778,-0.500000,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.800000,-0.125000,-0.125000,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.500000,-0.238095,-0.500000,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [53]:
# method to extract article title and content given an url 
def populateTitleAndBodyForURL(url):
    webPage = ul.urlopen(url)                                       
    soup = BeautifulSoup(webPage, "html.parser")   
    paras = ""

    # fetching title
    title = soup.find('title').text

    # fetching article content
    paragraphs = soup.findAll('p')     
    for ps in paragraphs:
        paras += (ps.text)
    
    return title, paras

In [75]:
# fetching title and content of the first 2000 article in the dataset
for i in range(len(data.loc[0:2000, 'url'])):
    title, paras = populateTitleAndBodyForURL(data.loc[i, 'url'])
    data.loc[i,'title'] = title
    data.loc[i,'articleContent'] = paras
    print(i)
data

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,title,articleContent
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,-0.350000,-0.600000,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,Amazon's Streaming Video Library Now a Little ...,Having trouble finding something to watch on A...
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,-0.118750,-0.125000,-0.100000,0.000000,0.000000,0.500000,0.000000,711,AP's Twitter to Begin Displaying Sponsored Tweets,The Associated Press is the latest news organi...
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,-0.466667,-0.800000,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,Apple's App Store Passes 40 Billion Downloads,It looks like 2012 was a pretty good year for ...
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,-0.369697,-0.600000,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,This Astronaut Is Rooting for Notre Dame Tonight,"When it comes to college football, NASA astron..."
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,-0.220192,-0.500000,-0.050000,0.454545,0.136364,0.045455,0.136364,505,New U-Verse Apps Simplify Sharing Photos and V...,LAS VEGAS â€” Sharing photos and videos on you...
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,-0.195000,-0.400000,-0.100000,0.642857,0.214286,0.142857,0.214286,855,BeeWi's Smart Toys Put Your Smartphone in Control,LAS VEGAS — RC toys have traded in their bulky...
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,-0.224479,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,556,BodyMedia Fitness Armband Gets a Fashion Upgrade,LAS VEGAS â€” If you shy away from fitness tra...
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,-0.242778,-0.500000,-0.050000,1.000000,0.500000,0.500000,0.500000,891,Canon PowerShot N Thinks Creatively So You Don...,LAS VEGAS â€” Love photos but never seem to ta...
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,-0.125000,-0.125000,-0.125000,0.125000,0.000000,0.375000,0.000000,3600,"Infographic: The Self-Driving, Ultra-Connected...",What will the car of the future look like? Wil...
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,-0.238095,-0.500000,-0.100000,0.000000,0.000000,0.500000,0.000000,710,Obama Defense Nominee Chuck Hagel Targeted by ...,Mere hours after President Barack Obama put fo...


In [78]:
# exporting to a csv (as the title and content extraction takes a long time)
data.to_csv("ContentExtracted.csv")

In [135]:
# loading data
newData = pd.read_csv("ContentExtracted.csv")
newData = newData.loc[0:2000, :]
newData

/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,...,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,title,articleContent
0,0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,...,-0.350000,-0.600000,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,Amazon's Streaming Video Library Now a Little ...,Having trouble finding something to watch on A...
1,1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,...,-0.118750,-0.125000,-0.100000,0.000000,0.000000,0.500000,0.000000,711,AP's Twitter to Begin Displaying Sponsored Tweets,The Associated Press is the latest news organi...
2,2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,...,-0.466667,-0.800000,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,Apple's App Store Passes 40 Billion Downloads,It looks like 2012 was a pretty good year for ...
3,3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,...,-0.369697,-0.600000,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,This Astronaut Is Rooting for Notre Dame Tonight,"When it comes to college football, NASA astron..."
4,4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,...,-0.220192,-0.500000,-0.050000,0.454545,0.136364,0.045455,0.136364,505,New U-Verse Apps Simplify Sharing Photos and V...,LAS VEGAS â€” Sharing photos and videos on you...
5,5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,...,-0.195000,-0.400000,-0.100000,0.642857,0.214286,0.142857,0.214286,855,BeeWi's Smart Toys Put Your Smartphone in Control,LAS VEGAS — RC toys have traded in their bulky...
6,6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,...,-0.224479,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,556,BodyMedia Fitness Armband Gets a Fashion Upgrade,LAS VEGAS â€” If you shy away from fitness tra...
7,7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,...,-0.242778,-0.500000,-0.050000,1.000000,0.500000,0.500000,0.500000,891,Canon PowerShot N Thinks Creatively So You Don...,LAS VEGAS â€” Love photos but never seem to ta...
8,8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,...,-0.125000,-0.125000,-0.125000,0.125000,0.000000,0.375000,0.000000,3600,"Infographic: The Self-Driving, Ultra-Connected...",What will the car of the future look like? Wil...
9,9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,...,-0.238095,-0.500000,-0.100000,0.000000,0.000000,0.500000,0.000000,710,Obama Defense Nominee Chuck Hagel Targeted by ...,Mere hours after President Barack Obama put fo...


In [136]:
newData.columns

Index(['Unnamed: 0', 'url', ' timedelta', ' n_tokens_title',
       ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words',
       ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs',
       ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords',
       ' data_channel_is_lifestyle', ' data_channel_is_entertainment',
       ' data_channel_is_bus', ' data_channel_is_socmed',
       ' data_channel_is_tech', ' data_channel_is_world', ' kw_min_min',
       ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max',
       ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg',
       ' self_reference_min_shares', ' self_reference_max_shares',
       ' self_reference_avg_sharess', ' weekday_is_monday',
       ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday',
       ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday',
       ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04',
       ' global_subjectivity',

In [139]:
# classifying articles as popular or not popular
le = preprocessing.LabelEncoder()
newData["article_popular"] = pd.Series(le.fit_transform(newData.loc[:,' shares']>=1400))

In [140]:
# title data tfidf vectorization
titleData = TfidfVectorizer(lowercase=True, strip_accents='unicode', stop_words=stopWords)
Xtitle = titleData.fit_transform(newData['title'])
titleColumns = [('title_' + feature) for feature in titleData.get_feature_names()]
Xtitle = pd.DataFrame(data = Xtitle.todense(), columns = titleColumns)

# Taking the top 500 best words
y = newData['article_popular']
selector = SelectKBest(chi2, k=500).fit(Xtitle, y)
XtitleTop500 = Xtitle.iloc[:,selector.get_support(indices=True)]
XtitleTop500

,title_10,title_11,title_128gb,title_17,title_200,title_22,title_24,title_36,title_360,title_38,...,title_wheels,title_white,title_william,title_winners,title_without,title_works,title_worth,title_worthy,title_yelp,title_yolo
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
bodyData = TfidfVectorizer(lowercase=True, strip_accents='unicode', stop_words=stopWords)
Xbody = bodyData.fit_transform(newData['articleContent'])
bodyCols = [('body_' + feature) for feature in bodyData.get_feature_names()]
Xbody = pd.DataFrame(data=Xbody.todense(), columns=bodyCols)

# Taking the top 500 best words
selector = SelectKBest(chi2, k=500).fit(Xbody, y)
XbodyTop500 = Xbody.iloc[:,selector.get_support(indices=True)]
XbodyTop500

,body_128gb,body_32gb,body_4sqdayimage,body_85th,body_abrams,body_activations,body_adblock,body_adele,body_adult,body_afdlips,...,body_white,body_wi,body_wikipedia,body_winbot,body_woof,body_works,body_yec,body_yolo,body_yuan,body_zamboni
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.049121,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.026533,0.035762,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [143]:
# Join initial DataFrame Xraw and features extracted from body and title
X = pd.concat([newData, XtitleTop500, XbodyTop500], axis=1)
X.drop([' shares','article_popular','title', 'url', 'articleContent'], inplace=True, axis=1)
X

,Unnamed: 0,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,body_white,body_wi,body_wikipedia,body_winbot,body_woof,body_works,body_yec,body_yolo,body_yuan,body_zamboni
0,0,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,1,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,2,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,3,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,4,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,5,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.000000,0.049121,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,6,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,7,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.026533,0.035762,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,8,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,9,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [144]:
# Standarization using Standard scaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()
Xnew = scaler.fit_transform(X)
XScaled = pd.DataFrame(Xnew, columns=X.columns)
Xtrain, Xtest, ytrain, ytest = train_test_split(XScaled, y, test_size = 0.2, random_state = 10)

/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [149]:
# optimize a logistic regression function by using a grid parameter search
logistic = LogisticRegression()
params = {'penalty': ['l1', 'l2'], 
          'C' : np.logspace(-10, 10, 10)}
tuning = GridSearchCV(logistic, params, cv=5)
tuning.fit(X,y)
print("Training Accuracy : ", tuning.best_score_)
tuning.best_estimator_.fit(Xtrain,ytrain)
print("Test Accuracy : ", tuning.score(Xtest,ytest))

/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increa

Training Accuracy :  0.7031484257871065
Test Accuracy :  0.7107231920199502


In [150]:
rf = RandomForestClassifier(random_state=37)
rf.fit(Xtrain, ytrain)
print("Test Accuracy : ", accuracy_score(ytest, rf.predict(Xtest)))

Test Accuracy :  0.5760598503740648


In [151]:
# Standarization using MinMax scaler
minMaxScaler = MinMaxScaler()
Xnew = minMaxScaler.fit_transform(X)
XScaled = pd.DataFrame(Xstd1, columns=X.columns)
Xtrain, Xtest, ytrain, ytest = train_test_split(XScaled, y, test_size = 0.2, random_state = 10)

/Users/dhivyaslh/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [153]:
mnb = MultinomialNB()
mnb.fit(Xtrain,ytrain)
print("Test accuracy : ", accuracy_score(ytest, mnb.predict(Xtest)))

Test accuracy :  0.7955112219451371
